In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from scipy.stats import entropy
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.decomposition import PCA


# 1. Carregamento dos dados
A planilha "tabela.tsv" foi importada via biblioteca pandas, com separador por tabulação.

Usei o skiprows para ignorar a primeira linha e remover o "# Constructed from biom file".

In [4]:
df = pd.read_csv("./dados/tabela.tsv", sep="\t", skiprows=1)

In [5]:
df.head(3)

,#OTU ID,16S-Amostra1,16S-Amostra2,taxonomy
0,321501a64794e926be7b961674c35a0f,0.0,1170.0,d__Bacteria; p__Actinobacteriota; c__Actinomyc...
1,2c47537f334048fd1472da325f00561d,0.0,316.0,d__Bacteria; p__Actinobacteriota; c__Actinomyc...
2,4a1547af0bbae2aff8f3222d2fba2102,179.0,0.0,d__Bacteria; p__Firmicutes_A; c__Clostridia; o...


## ---

# 2. Cálculo da Abundância Relativa dos gêneros por amostra

O gênero foi obtido da coluna "taxonomy", sendo aqueles que se inicial por "g__"

In [6]:
df["Genero"] = df["taxonomy"].str.extract(r"g__([^;]*)")

Defini os valores sem informações de gênero como "Desconhecido"

In [7]:
df["Genero"] = df["Genero"].fillna("Desconhecido")

Optei por remover os sufixos "_A" e "_B" dos gêneros para padronizar a nomenclatura e evitar confusões. Presumi que ambos pertencem ao mesmo gênero e que essas variações representam formas distintas dentro dele, sem, no entanto, indicarem uma classificação formal de espécie.

In [8]:
df["Genero"] = df["Genero"].str.replace(r"_\w$", "", regex=True)

Os dados quantitativos (provavelmente reads) de cada amostra foram somados e agrupados por gênero

In [9]:
genero_abundancia = df.groupby("Genero")[["16S-Amostra1", "16S-Amostra2"]].sum()

A abundância relativa foi calculada através do percentual quantitativo de cada gênero em relação ao total. Optei por arredondar para 2 casas decimais visando facilitar a visualização

In [10]:
genero_abundancia_relativa = (genero_abundancia.div(genero_abundancia.sum()) * 100).round(2)
genero_abundancia_relativa.columns = ["Amostra 1 (%)", "Amostra 2 (%)"]
genero_abundancia_relativa = genero_abundancia_relativa.reset_index() 

pd.set_option('display.max_rows', None) #OBS. Esse comando permite desabilitar o número máximo de linhas mostradas
genero_abundancia_relativa


,Genero,Amostra 1 (%),Amostra 2 (%)
0,Acetatifactor,0.80,0.00
1,Actinopolyspora,0.00,1.17
2,Agathobacter,3.35,0.00
3,Agathobaculum,1.48,0.00
4,Akkermansia,0.19,0.00
5,Alicyclobacillus,0.00,0.21
6,Alistipes,0.95,0.00
7,Anaerobutyricum,0.49,0.00
8,Anaerostipes,1.33,0.00
9,Bacteroides,4.72,0.00


Criação de uma pasta para os resultados

In [13]:
os.makedirs("./resultados", exist_ok=True)

A tabela foi salva em "genero_abundancia_relativa.tsv"

In [14]:
genero_abundancia_relativa.to_csv("./resultados/genero_abundancia_relativa.tsv", sep="\t", index=False)

## ---

# 3. Elaboração de visualizações gráficas

## Gráfico de Barras Empilhadas

O seguinte código abaixo busca desenvolver um gráfico de barras empilhadas considerando o gênero e a abundância relativa calculada anteriormente para cada uma das amostras. Optei por fazer um gráfico interativo para melhor visualização de cada gênero e sua respectiva abundância de forma individual.

Caso seja do interesse salvar de forma não interativa, basta não comentar a última linha de código

In [15]:
amostras = ["Amostra 01", "Amostra 02"]

cores = px.colors.qualitative.Plotly

trace_list = []
for i, genero in enumerate(genero_abundancia_relativa["Genero"].unique()):
    valores = genero_abundancia_relativa.loc[
        genero_abundancia_relativa["Genero"] == genero, ["Amostra 1 (%)", "Amostra 2 (%)"]
    ].values.flatten()

    trace = go.Bar(
        x=amostras,
        y=valores,
        name=genero, 
        hovertemplate='<b>%{x}</b><br>Gênero: ' + genero + '<br>Abundância Relativa: %{y:.2f}%',
        marker=dict(color=cores[i % len(cores)], line=dict(color="black", width=0.7)),
        hoverlabel=dict(namelength=0) 
    )
    trace_list.append(trace)

layout = go.Layout(
    title=dict(
        text="Gráfico de barras empilhadas sobre a Abundância Relativa de Gêneros nas Amostras",
        x=0.5, 
        font=dict(size=16, family="Arial, sans-serif", color="black")
    ),
    xaxis=dict(
        title="Amostras",
        tickfont=dict(size=14),
        showline=True,
        linecolor="black"
    ),
    yaxis=dict(
        title="Abundância Relativa (%)",
        tickfont=dict(size=14),
        showgrid=True,
        gridcolor="lightgrey",
        zeroline=True,
        zerolinecolor="black"
    ),
    barmode="stack",
    legend=dict(
        title="Gêneros",
        x=1.02,
        y=1,
        font=dict(size=12)
    ),
    plot_bgcolor="white",
    margin=dict(t=60, r=50, b=60, l=70),
)

fig = go.Figure(data=trace_list, layout=layout)
fig.show()


fig.write_html("./resultados/grafico_barras_empilhadas.html")
# fig.write_image("./resultados/grafico_barras_empilhadas.png") #Caso queira salvar o gráfico de forma não interativa

## Outros gráficos

Fiz também outras opções de gráfico para complementar a ilustração da Abundância Relativa.

## Heatmap

In [16]:
data = genero_abundancia_relativa[['Genero', 'Amostra 1 (%)', 'Amostra 2 (%)']].set_index('Genero').transpose()

fig2 = go.Figure(data=go.Heatmap(
    z=data.values,
    x=data.columns,
    y=data.index,
    colorscale='Blues',
    colorbar=dict(title="Abundância Relativa (%)"),
    text=data.values,
    hoverinfo='text',
    showscale=True,
    hovertemplate='<br>Gênero: %{x} ' + '<br>Abundância Relativa: %{z:.2f}%',
    hoverlabel=dict(namelength=0)
))

fig2.update_layout(
    title='Heatmap sobre a Abundância Relativa de Gêneros nas Amostras',
    xaxis_title='Gênero',
    yaxis_title='Amostras',
    xaxis=dict(showgrid=True, tickangle=45),
    yaxis=dict(tickvals=[0, 1], ticktext=['Amostra 1', 'Amostra 2']),
    height=300, 
    margin=dict(l=50, r=50, t=50, b=100), 
    xaxis_tickangle=45,
)

fig2.show()

fig2.write_html("./resultados/heatmap.html")
# fig2.write_image("./resultados/heatmap.png") #Caso queira salvar o gráfico de forma não interativa

## SankeyPlot

In [17]:
labels = ['Amostra 1', 'Amostra 2'] + list(genero_abundancia_relativa['Genero'].unique())
sources = []
targets = []
values = []
colors = []

color_scale = 'Blues'  

for index, row in genero_abundancia_relativa.iterrows():
    for i, amostra in enumerate(['Amostra 1 (%)', 'Amostra 2 (%)']):
        if row[amostra] > 0:
            sources.append(i) 
            targets.append(labels.index(row['Genero']) + 2) 
            values.append(row[amostra])
            colors.append(row[amostra])

fig3 = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
        color="darkgreen",
        hovertemplate=('Gênero: ' + '%{label} <br>' + 'Abundância Relativa: %{value:.2f}%'),
        hoverlabel=dict(namelength=0)
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=["rgba(0, 255, 0, {})".format(val/100) for val in colors]
    )
))

fig3.update_layout(
    title_text="Sankey Plot sobre a Abundância Relativa de Gêneros nas Amostras",
    font_size=12,
    title_font_size=16,
    title_x=0.5,  
    title_y=0.95, 
    dragmode=False,
    hovermode='closest',
    showlegend=False, 
    modebar=dict(
        remove=['zoom', 'pan', 'reset', 'zoomIn', 'zoomOut', 'hoverCompare', 'toggleSpikelines', 'toImage']
    ) 
)

fig3.show()

fig3.write_html("./resultados/sankey.html")
# fig3.write_image("./resultados/hsankey.png") #Caso queira salvar o gráfico de forma não interativa

## ---

# 4. Criação de um relatório automatizado

Optei por gerar um relatório automatizado em HTML para a melhor apresentação e divulgação dos resultados.

Gerar tabela em HTML formatada

In [18]:
tabela_html = genero_abundancia_relativa.to_html(index=False, classes='table table-striped table-responsive text-center', border=0)

Criar relatório em HTML

In [19]:
html_content = f'''
<!DOCTYPE html>
<html lang="pt-br">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Processo seletivo - Bioinformática GoGenetic (Relatório)</title>
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.3.0/css/bootstrap.min.css">
    <style>
        .container {{
            max-width: 800px;
            margin: auto;
        }}
        table {{
            width: 100%;
            text-align: center;
        }}
    </style>
</head>
<body>
    <div class="container">
        <h1 class="text-center mt-4">Processo seletivo - Bioinformática GoGenetic (Relatório)</h1>
        <hr>
        <h2 class="text-center">Tabela de Abundância Relativa</h2>
        {tabela_html}  <!-- Tabela de dados -->
        <hr>
        <h2 class="text-center">Gráficos de Abundância</h2>
        <div id="grafico"></div>
        <hr>
        <div id="grafico2"></div>
        <hr>
        <h2 class="text-center"></h2>
        <div id="grafico3"></div>  <!-- Novo gráfico Sankey -->
    </div>
    <script>
        var grafico = {fig.to_json()};  // Gráfico 1
        Plotly.newPlot('grafico', grafico.data, grafico.layout);

        var grafico2 = {fig2.to_json()};  // Gráfico 2
        Plotly.newPlot('grafico2', grafico2.data, grafico2.layout);

        var grafico3 = {fig3.to_json()};  // Gráfico Sankey
        Plotly.newPlot('grafico3', grafico3.data, grafico3.layout);  // Exibindo o gráfico Sankey
    </script>
    <div class="container text-center">
    <h2>Resultados</h2>
        <h3 style="font-weight: normal;">A Amostra 1 apresenta uma maior diversidade de gêneros, com várias categorias exibindo porcentagens mais bem distribuídas. Sua diversidade é significativamente maior, sugerindo uma comunidade microbiana mais equilibrada e variada.</h3>
        <h3 style="font-weight: normal;">A Amostra 2 é dominada principalmente pelo gênero Nocardioides (com Abundância Relativa de 52,45%), seguindo pelos gêneros Spirillospora (13,20%) e Trebonia (11,24%). Conjuntamente, outros gêneros aparecem com menores frequências. Seu perfil pode indicar uma possível seleção ambiental ou condições específicas favorecendo determinados microrganismos.</h3>  
        <h3 style="font-weight: normal;">Alguns gêneros são exclusivos de cada amostra, sem sobreposição entre elas. Por exemplo, Faecalibacterium (8,79%), Prevotella (8,30%) e Phocaeicola (7,12%) são abundantes na Amostra 1, mas ausentes na Amostra 2. A presença de um percentual considerável de gêneros desconhecidos em ambas as amostras (6,70% na Amostra 1 e 1,85% na Amostra 2) sugere que há componentes ainda não caracterizados na microbiota analisada.</h3> 
    </div>
</body>
</html>
'''


Salvar o relatório como um arquivo HTML

In [20]:
report_path = "./resultados/relatorio.html"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(html_content)